In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
import os

In [4]:
def write_doc(dir_path, doc_text, count):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    f = open('{}/doc{}.txt'.format(dir_path, str(count)), 'w')
    f.write(doc_text)
    f.close()

In [5]:
# Заветы Ильича
def get_articles(start, count, dir_path):
    cnt = 0
    for i in range(count):
        page = requests.get('http://zavety-i.ru/?module=articles&action=view&id='+str(start+i))
        html = page.text
        soup = BeautifulSoup(html, 'lxml')
        text = []
        text_tags = soup('div', {'class' :"full-news"})
        if text_tags != []:
            for tag in text_tags[0]:
                if tag.name == 'p':
                    text.append(tag.string)
        if text != []:
            author = 'Noname'
            title = soup.find('h2', {'class' :"title"}).string
            date = soup.find('span', {'class' : "date"}).string.split('.')
            date_ = '.'.join([date[2], date[1], date[0]])
            topic = ''
            url = 'http://zavety-i.ru/?module=articles&action=view&id='+str(start+i)
            if None not in ([author, title, date_, url]):
                formated = '@au {}\n@ti {}\n@da {}\n@topic {}\n@url {}\n{}'.format(author, title, date_, 
                                                                                   topic, url, ' '.join(text))
                write_doc(dir_path, formated, cnt)
                cnt += 1
    print(str(cnt)+' articles')

In [90]:
# Молодежная газета
def get_more_articles(count, dir_path):
    cnt = 15
    page_number = 1
    count = count+15
    
    while cnt < count and page_number < 56:
        parent_page = requests.get('http://mgazeta.com/category/tak-i-zhivem/?PAGEN_1='+str(page_number)).text
        parent_soup = BeautifulSoup(parent_page, 'lxml')
        links = parent_soup.findAll('a', {'class' :"item-prev-image"})
        page_number += 1
        for link in links:
            if cnt >= count:
                break
            url = 'http://mgazeta.com'+link['href']
            article_page = requests.get(url).text
            article_soup = BeautifulSoup(article_page, 'lxml')
            author = article_soup.find('span', {'class' : "news-property prop-AUTORS "})
            if author:
                author = author.string.strip()
            else:
                author = 'Noname'
            title = article_soup.find('h1').string
            date = article_soup.find('span', {'class' : "news-date-time"}).string.split()[0]
            topic = article_soup.find('div', {'class': "top-section-name"}).string
            text = []
            text_tags1 = article_soup('div', {'class' :"news-detail article-detail"})
            if text_tags1 != []:
                for tag in text_tags1[0]:
                    if tag.name == 'p' and tag.string:
                        text.append(tag.string) 
            text_tags2 = article_soup('div', {'class' :"news-text js-mediator-article"})
            if text_tags2 != []:
                for tag in text_tags2[0]:
                    if tag.string:
                        text.append(tag.string.strip())
            if None not in (author, title, date, topic) and text != []:
                formated = '@au {}\n@ti {}\n@da {}\n@topic {}\n@url {}\n{}'.format(author, title, date, 
                                                                                           topic, url, ' '.join(text))
                write_doc(dir_path, formated, cnt)
            cnt += 1
           
    print(str(cnt)+' articles')

In [91]:
get_more_articles(700, 'articles_collection')

673 articles
